In [0]:
!pip -qq install fastai2

     |████████████████████████████████| 184kB 9.0MB/s 


In [0]:
from fastai2.vision.all import *
from fastai2.vision.widgets import *

In [0]:
PATH = untar_data(URLs.PETS)

To use Pathlib

In [0]:
Path.BASE_PATH = PATH

(#2) [Path('annotations'),Path('images')]

In [0]:
data = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460), # Crop full width or height. It is applied to each individual image before it is copied to the GPU. 
                                        # It is used to ensure all images are the same size. On the training set, the crop area is chosen randomly. 
                                        # On the validation set, the center square of the image is always chosen.
                 batch_tfms=aug_transforms(size=224, min_scale=0.75)) # Random crop and augment. It is applied to a batch all at once on the GPU, which means it's fast. 
                                                                      # On the validation set, only the resize to the final size needed for the model is done here. 
                                                                      # On the training set, the random crop and any other augmentation is done first.
dls = data.dataloaders(PATH/'images')

In [0]:
learn_inf = load_learner('export.pkl')

In [0]:


btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [0]:
img = PILImage.create(btn_upload.data[-1])

In [0]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

Output()

In [0]:
pred, pred_idx, probs = learn_inf.predict(img)

In [0]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

Label(value='Prediction: Bombay; Probability: 0.4010')

In [0]:
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [0]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)